# Приложение для чтения книг
***Цель исследования:***  анализ информации о книгах, издателях авторах, пользовательских обзорах, содержащихся в приобретенной базе данных книг для формирования предложения по новому приложению для чтения книг.

***Описание исходных данных:*** 

1) Таблица `books`: содержит данные о книгах.
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

2) Таблица `authors`: содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

3) Таблица `publishers`: содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

4) Таблица `ratings`: содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

5) Таблица `reviews`: содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Схема данных: таблица `books` связана с:
- `ratings` и `reviews` через внешний для них ключ `book_id`;
- `authors` через свой первичный ключ `author_id`;
- `publishers` через свой первичный ключ `publisher_id`.


***Шаги исследования:***
1. Загрузка и исследование данных.
2. Количество книг, вышедших после 1 января 2000 года.
3. Количество обзоров и средняя оценка для каждой книги.
4. Издательство, которое выпустило наибольшее число книг
5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

## Загрузка и исследование данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': '-censored-', # пароль
'host': '-censored-',
'port': '-censored-', # порт подключения
'db': '-censored-'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# функция для загрузки данных из sql базы
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine) 

In [3]:
# вывод первых строк таблицы с книгами
sql_b = '''
        SELECT *
        FROM books
        LIMIT 5
        '''
select(sql_b)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# подсчет количества строки и уникальных значений
sql_b_1 = '''
        SELECT  COUNT(*) AS rows,
                COUNT(DISTINCT book_id) AS books_uniq,
                COUNT(DISTINCT title) AS titles_uniq,
                COUNT(DISTINCT author_id) AS author_uniq,
                COUNT(DISTINCT publisher_id) AS publisher_uniq,
                MAX(publication_date) AS latest_date,
                MIN(publication_date) AS earliest_date
        FROM books
        '''
select(sql_b_1)

,rows,books_uniq,titles_uniq,author_uniq,publisher_uniq,latest_date,earliest_date
0,1000,1000,999,636,340,2020-03-31,1952-12-01


In [5]:
# подсчет количества строки и уникальных значений
sql_b_dup = '''
        SELECT  title,
                COUNT(book_id)
        FROM books
        GROUP BY title
        HAVING COUNT(book_id)>1
        '''
select(sql_b_dup)

,title,count
0,Memoirs of a Geisha,2


In [6]:
# подсчет количества строки и уникальных значений
sql_b_dup1 = '''
        SELECT  *
        FROM books
        WHERE title =  'Memoirs of a Geisha'
        '''
select(sql_b_dup1)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


В данных содержится информация о 1000 книг, одна из которых переиздавалась дважды, написанных 636 авторами, выпущенных с 1952 по 2020 год, в 340 издательствах.

In [7]:
# вывод первых строк таблицы с авторами
sql_a = '''
        SELECT *
        FROM authors
        LIMIT 5
        '''
select(sql_a)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
# подсчет количества строки и уникальных значений
sql_a_1 = '''
        SELECT  COUNT(*) AS rows,
                COUNT(DISTINCT author) AS author_uniq
        FROM authors
        '''
select(sql_a_1)

,rows,author_uniq
0,636,636


Количество уникальных авторов совпадает с их уникальным количеством в таблице с книгами

In [9]:
# вывод первых строк таблицы с издательствами
sql_p = '''
        SELECT *
        FROM publishers
        LIMIT 5
        '''
select(sql_p)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
# подсчет количества строки и уникальных значений
sql_p_1 = '''
        SELECT  COUNT(*) AS rows,
                COUNT(DISTINCT publisher) AS publisher_uniq
        FROM publishers
        '''
select(sql_p_1)

,rows,publisher_uniq
0,340,340


Количество уникальных издательств совпадает с их уникальным количеством в таблице с книгами

In [11]:
# вывод первых строк таблицы с пользовательскими оценками
sql_rt = '''
        SELECT *
        FROM ratings
        LIMIT 5
        '''
select(sql_rt)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
# подсчет количества строки и уникальных значений
sql_rt_1 = '''
        SELECT  COUNT(*) AS rows,
                COUNT(DISTINCT rating_id) AS rating_id_uniq,
                COUNT(DISTINCT book_id) AS book_id_uniq,
                COUNT(DISTINCT username) AS username_uniq,
                MAX(rating) AS max_rating,
                MIN(rating) AS min_rating
        FROM ratings
        '''
select(sql_rt_1)

,rows,rating_id_uniq,book_id_uniq,username_uniq,max_rating,min_rating
0,6456,6456,1000,160,5,1


Диапазон оценки в рейтинге от 1 до 5. Рейтинг представлен на основе мнения 160 уникальных пользователей для того же количества книг, которое содержится в таблице с книгами. 

In [13]:
# вывод первых строк таблицы с обзорами
sql_rw = '''
        SELECT *
        FROM reviews
        LIMIT 5
        '''
select(sql_rw)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [14]:
# подсчет количества строки и уникальных значений
sql_rw_1 = '''
        SELECT  COUNT(*) AS rows,
                COUNT(DISTINCT review_id) AS review_id_uniq,
                COUNT(DISTINCT book_id) AS book_id_uniq,
                COUNT(DISTINCT username) AS username_uniq,
                COUNT(DISTINCT text) AS text_uniq
        FROM reviews
        '''
select(sql_rw_1)

,rows,review_id_uniq,book_id_uniq,username_uniq,text_uniq
0,2793,2793,994,160,2793


Такое же количество пользователей, которое оставило оценки, написало 2793 уникальных обзора на 994 книги.

In [15]:
# вывод сгруппированных показателей
sql_comb = '''
        SELECT COUNT(*) AS rows,
                COUNT(DISTINCT b.book_id) AS book_id_uniq,
                COUNT(DISTINCT b.title) AS title_uniq,
                COUNT(DISTINCT a.author_id) AS author_id_uniq,
                COUNT(DISTINCT p.publisher_id) AS publisher_id_uniq,
                COUNT(DISTINCT rt.username) AS username_uniq,
                COUNT(DISTINCT rw.review_id) AS review_id_uniq,
                COUNT(DISTINCT rw.text) AS text_uniq,
                MAX(b.publication_date) AS latest_date,
                MIN(b.publication_date) AS earliest_date
        FROM books AS b
        JOIN authors AS a ON b.author_id = a.author_id
        JOIN publishers AS p ON b.publisher_id = p.publisher_id
        JOIN ratings AS rt ON b.book_id = rt.book_id
        JOIN reviews AS rw ON b.book_id = rw.book_id
        '''
select(sql_comb)

,rows,book_id_uniq,title_uniq,author_id_uniq,publisher_id_uniq,username_uniq,review_id_uniq,text_uniq,latest_date,earliest_date
0,26167,994,993,630,337,160,2793,2793,2020-03-31,1952-12-01


In [16]:
# вывод сгруппированных показателей
sql_comb = '''
        SELECT COUNT(*) AS rows,
                COUNT(DISTINCT b.book_id) AS book_id_uniq,
                COUNT(DISTINCT b.title) AS title_uniq,
                COUNT(DISTINCT a.author_id) AS author_id_uniq,
                COUNT(DISTINCT p.publisher_id) AS publisher_id_uniq
               
        FROM books AS b
        JOIN authors AS a ON b.author_id = a.author_id
        JOIN publishers AS p ON b.publisher_id = p.publisher_id
      
        '''
select(sql_comb)

,rows,book_id_uniq,title_uniq,author_id_uniq,publisher_id_uniq
0,1000,1000,999,636,340


***Вывод:***
- из 1000 уникальных id книг вся информация представлена только для 994 книг, у 6 отсутсвуют обзоры;
- книги написаны 636 авторами и выпущены в 340 издательствах в период с 1 декабря 1952 года по 31 марта 2020 года;
- 160 пользователей дали на книги 6456 оценок и 2793 обзора.

## Количество книг, вышедших после 1 января 2000 года

In [17]:
query1 = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-02 00:00:00'
        '''
select(query1)

,count
0,819


***Вывод:***

Несмотря на то, что в данных содержится информация о книгах, выпущенных с 1952 года, 82% книг выпущено после 01.01.2000 года.

## Количество обзоров и средняя оценка для каждой книги

In [25]:
query2 = '''
        SELECT b.book_id,
                b.title,
                COUNT(DISTINCT rw.review_id) AS nmb_reviews,
                ROUND(AVG(rt.rating), 2) AS avg_rating
        FROM books AS b
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        LEFT JOIN reviews AS rw ON b.book_id = rw.book_id  
        GROUP BY b.book_id, b.title
        ORDER BY nmb_reviews DESC, avg_rating DESC
        
        '''
select(query2)



,book_id,title,nmb_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


In [19]:
# средний рейтинг в зависимости от количества обзоров
query2 = '''
            WITH top AS (SELECT b.book_id,
                                b.title,
                                COUNT(DISTINCT rw.review_id) AS nmb_reviews,
                                ROUND(AVG(rt.rating), 2) AS rating_avg
                        FROM books AS b
                        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                        LEFT JOIN reviews AS rw ON b.book_id = rw.book_id  
                        GROUP BY b.book_id, b.title
                        ORDER BY rating_avg DESC
            )
            SELECT nmb_reviews,
                    AVG(rating_avg)
            FROM top
            GROUP BY nmb_reviews
            ORDER BY avg DESC
        '''
select(query2)



,nmb_reviews,avg
0,6,3.996000
1,5,3.957500
2,0,3.945000
3,2,3.914369
4,4,3.908375
5,3,3.862238
6,1,3.815135
7,7,3.660000


In [20]:
query4_1 = '''
            SELECT *
            FROM books AS b
            JOIN authors AS a ON b.author_id = a.author_id
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.book_id = 948
        '''
select(query4_1)

,book_id,author_id,title,num_pages,publication_date,publisher_id,author_id,author,publisher_id,publisher
0,948,554,Twilight (Twilight #1),501,2006-09-06,176,554,Stephenie Meyer,176,Little Brown and Company


***Вывод:***
Наибольшее количество обзоров на книгу Twilight (Twilight #1). На 2 и 3 местах (с учетом более высокого среднего рейтинга) - 2 книги о Гарри Поттере. В целом, очевидная взаимосвязь среднего рейтинга и количества обзоров не наблюдается.

## Издательство, которое выпустило наибольшее число книг (толще 50 страниц)

In [21]:
query3 = '''
        SELECT p.publisher_id,
                p.publisher,
                COUNT(DISTINCT b.book_id) AS nmb_books
        FROM publishers AS p
        LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE b.num_pages >50
        GROUP BY p.publisher_id,p.publisher
        ORDER BY nmb_books DESC
        LIMIT 1
        '''
select(query3)



,publisher_id,publisher,nmb_books
0,212,Penguin Books,42


***Вывод:***
Издательство Penguin Books выпустило больше всего (42 шт) уникальных книг, толщиной более 50 страниц.

## Автор с самой высокой средней оценкой книг (только книги с 50 и более оценками)

In [22]:
query4 = '''
            SELECT DISTINCT author,
                    ROUND(AVG(avg_rating) OVER(PARTITION BY author),2) AS authors_rating
            FROM 
                (SELECT a.author,
                        b.book_id,
                        AVG(rt.rating) AS avg_rating,
                        COUNT(rt.rating_id) AS nmb_ratings
                FROM books AS b
                LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                LEFT JOIN authors AS a ON b.author_id = a.author_id  
                GROUP BY b.book_id, a.author) AS auth
            WHERE nmb_ratings > 50
            ORDER BY authors_rating DESC
            LIMIT 1
        '''
select(query4)

,author,authors_rating
0,J.K. Rowling/Mary GrandPré,4.28


In [23]:
query4_1 = '''
            SELECT *
            FROM books AS b
            JOIN authors AS a ON b.author_id = a.author_id
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE a.author = 'J.K. Rowling/Mary GrandPré'
        '''
select(query4_1)

,book_id,author_id,title,num_pages,publication_date,publisher_id,author_id,author,publisher_id,publisher
0,298,236,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690,2004-09-13,253,236,J.K. Rowling/Mary GrandPré,253,Scholastic
1,299,236,Harry Potter and the Chamber of Secrets (Harry...,341,1999-06-02,24,236,J.K. Rowling/Mary GrandPré,24,Arthur A. Levine Books / Scholastic Inc.
2,300,236,Harry Potter and the Half-Blood Prince (Harry ...,652,2006-09-16,255,236,J.K. Rowling/Mary GrandPré,255,Scholastic Inc.
3,301,236,Harry Potter and the Order of the Phoenix (Har...,870,2004-09-01,255,236,J.K. Rowling/Mary GrandPré,255,Scholastic Inc.
4,302,236,Harry Potter and the Prisoner of Azkaban (Harr...,435,2004-05-01,255,236,J.K. Rowling/Mary GrandPré,255,Scholastic Inc.


***Вывод:***
Самая овысокая средняя оценка книг у Дж.Роулинг с книгами о Гарри Поттере.

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [24]:
query5 = '''
            SELECT CAST(COUNT(review_id) AS float)/CAST(COUNT(DISTINCT username) AS float) AS avg_numb_of_reviews
            FROM reviews
            WHERE username IN (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id)>50)
            
        '''
select(query5)

,avg_numb_of_reviews
0,24.333333


***Вывод:***
Среднее количество обзоров у тех пользователей, кто поставил больше 50 оценок, составляет 24,33.

## Общие выводы

- В исходных данных содержится информация 1000 уникальных id книг, среди которых одна переиздавалась дважды. Только для 994 книг из 1000 представлена вся информация об авторах, издательствах, рейтингах и оценках. У 6 книг недостает обзоров;
- книги издавались в период с 1952 по 2020 годы, но 82% из них изданы после 1 января 2000 года;
- 160 пользователей оставили 6 456 оценок и 2 793 обзора. Только 6 пользователей (4%) оставили более 50 оценок (это минимум 5% всех оценок), среднее количество обзоров, оставленных ими же,  = 24.33 (тоже около 5% от всех обзоров). 
- Наибольшее количество обзоров у книги Twilight (Twilight #1) - 7 шт. На 2 и 3 местах (с учетом более высокого среднего рейтинга) - 2 книги о Гарри Поттере, у которых по 6 обзоров. В целом, очевидная взаимосвязь среднего рейтинга и количества обзоров не наблюдается.
- Из 340 издательств наибольшее количество книг (42 шт) выпустило Penguin Books, которое не выпускало самые популярные по рейтингу и обзорам книги о Гарри Поттере и Сумерках;
- Из 636 авторов, которых оценили более 50 раз, самая высокая оценка (4.28) у Джоан Роулинг.

Из анализа следует, что в данных содержатся в основном книги, изданные в 21 веке, наиболее популярными из которых являются книги о Гарри Поттере и Сумерках, что скорее свидетельствует о более молодой аудитории пользователей. 